# 加载数据集

In [9]:
import torch
from datasets import load_dataset


#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset('csv',data_files='./train.csv',split='train')

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label


dataset = Dataset('train')

len(dataset), dataset[0]

Using custom data configuration default-7fc9a321a99182ac
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-7fc9a321a99182ac/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


(9600,
 ('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  1))

# 加载tokenizer

In [2]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [21]:
from transformers import AutoTokenizer, AutoModel

# 加载字典和分词工具
tokenizer = AutoTokenizer.from_pretrained("./longformer_zh")

model = AutoModel.from_pretrained('./longformer_zh')
# token

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [22]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("schen/longformer-chinese-base-4096")

model = AutoModelForPreTraining.from_pretrained("schen/longformer-chinese-base-4096")

Downloading: 100%|██████████| 88.0/88.0 [00:00<00:00, 79.9kB/s]
Downloading: 100%|██████████| 839/839 [00:00<00:00, 705kB/s]
Downloading: 100%|██████████| 128k/128k [00:01<00:00, 130kB/s]  
Downloading: 100%|██████████| 112/112 [00:00<00:00, 114kB/s]
Downloading: 100%|██████████| 403M/403M [05:29<00:00, 1.28MB/s] 
Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertForPreTraining: ['bert.encoder.layer.8.attention.self.key_global.bias', 'bert.encoder.layer.5.attention.self.query_global.bias', 'bert.encoder.layer.0.attention.self.key_global.weight', 'bert.encoder.layer.0.attention.self.value_global.bias', 'bert.encoder.layer.3.attention.self.value_global.bias', 'bert.encoder.layer.8.attention.self.query_global.bias', 'bert.encoder.layer.2.attention.self.key_global.bias', 'bert.encoder.layer.11.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.key_global.bias', 'bert.encoder.layer.1.attention.self.query_globa

In [23]:
tokenizer

PreTrainedTokenizerFast(name_or_path='schen/longformer-chinese-base-4096', vocab_size=21128, model_max_len=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

# 定义批处理函数

In [4]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字，就是编码后的词
    #attention_mask:是补零的位置是0,其他位置是1。pad的位置是0,其他位置是1
    #token_type_ids：第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

In [36]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = tokenizer.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=4096,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字，就是编码后的词
    #attention_mask:是补零的位置是0,其他位置是1。pad的位置是0,其他位置是1
    #token_type_ids：第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

# 定义数据加载器

In [37]:
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

# 取第一批数据
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

600


(torch.Size([16, 4096]),
 torch.Size([16, 4096]),
 torch.Size([16, 4096]),
 tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0]))

# 加载BERT中文模型

In [7]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)
# 16是batch_size，对应数据中的16句话
# 500是数据分词的长度，对数据编码的时候，指定每一句话编码成500个词的长度
# 768是词编码的维度，也就是把每一个词编码成768维度的向量
out.last_hidden_state.shape

Downloading: 100%|██████████| 393M/393M [05:18<00:00, 1.29MB/s] 
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 500, 768])

In [38]:
#加载预训练模型
pretrained = AutoModelForPreTraining.from_pretrained("schen/longformer-chinese-base-4096")

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)
# 16是batch_size，对应数据中的16句话
# 500是数据分词的长度，对数据编码的时候，指定每一句话编码成500个词的长度
# 768是词编码的维度，也就是把每一个词编码成768维度的向量
# out.last_hidden_state.shape

Some weights of the model checkpoint at schen/longformer-chinese-base-4096 were not used when initializing BertForPreTraining: ['bert.encoder.layer.8.attention.self.key_global.bias', 'bert.encoder.layer.5.attention.self.query_global.bias', 'bert.encoder.layer.0.attention.self.key_global.weight', 'bert.encoder.layer.0.attention.self.value_global.bias', 'bert.encoder.layer.3.attention.self.value_global.bias', 'bert.encoder.layer.8.attention.self.query_global.bias', 'bert.encoder.layer.2.attention.self.key_global.bias', 'bert.encoder.layer.11.attention.self.key_global.weight', 'bert.encoder.layer.6.attention.self.key_global.bias', 'bert.encoder.layer.1.attention.self.query_global.bias', 'bert.encoder.layer.4.attention.self.value_global.weight', 'bert.encoder.layer.8.attention.self.query_global.weight', 'bert.encoder.layer.11.attention.self.query_global.bias', 'bert.encoder.layer.5.attention.self.key_global.bias', 'bert.encoder.layer.5.attention.self.key_global.weight', 'bert.encoder.layer

In [35]:
print(out.hidden_states)

None


# 定义下游任务模型

In [34]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 这里只定义了一个全连接网络
        # 做n分类的话，只需要把2改成n
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 计算过程
        # 1.先拿预训练模型做一个计算，抽取数据中的特征
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        # 2.把抽取出来的特征放到全连接网络中，并且这个特征只需要第0个词的特征
        # [CLS]用于分类任务，并且出现在bert输出的第 0 index
        out = self.fc(out.hidden_states[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

TypeError: 'NoneType' object is not subscriptable

# 训练下游任务

In [9]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 300:
        break

/home/chenli/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0 0.7007757425308228 0.5
5 0.6961469650268555 0.5
10 0.6280856132507324 0.75
15 0.6409363746643066 0.625
20 0.5809353590011597 0.75
25 0.5867862701416016 0.75
30 0.6139634847640991 0.625
35 0.563926100730896 0.6875
40 0.5806879997253418 0.75
45 0.47617384791374207 1.0
50 0.5543134212493896 0.875
55 0.45374664664268494 0.9375
60 0.44496291875839233 0.9375
65 0.4599880874156952 0.875
70 0.492148220539093 0.8125
75 0.5374161005020142 0.75
80 0.47261425852775574 0.875
85 0.5308655500411987 0.6875
90 0.5169926881790161 0.875
95 0.4295022785663605 0.9375
100 0.4518877863883972 0.875
105 0.5008432865142822 0.8125
110 0.4429056644439697 1.0
115 0.45786380767822266 0.875
120 0.3863544464111328 1.0
125 0.4485105276107788 0.875
130 0.50847327709198 0.8125
135 0.5336365699768066 0.75
140 0.517758846282959 0.8125
145 0.454900860786438 0.9375
150 0.5203752517700195 0.8125
155 0.4101777970790863 1.0
160 0.5227699279785156 0.8125
165 0.44081419706344604 1.0
170 0.45648327469825745 0.8125
175 0.5407354

# 总结
在训练的过程中可以发现，正确率已经达到80 90的样子。这个就是使用 BERT 预训练模型抽取特征所展示出来的威力。以往在这种自然语言上的数据集上面，想要做一个文本分类的任务，想要训练到80 90的正确率，这个训练量是非常大的，模型往往很难收敛。但是从这里发现，使用BERT做预训练模型抽取特征，然后再做下游任务的一个迁移学习。可以在非常短的时间内以非常快的速度就能够达到很高的正确率。

# 测试

In [11]:
#测试
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 50:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()

Using custom data configuration default-13ed2499b3a51032
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-13ed2499b3a51032/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0.869375
